In [ ]:
##libraries
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import shutil

import tensorflow as tf

In [4]:
##project constant
RAW_DIR="data/Art"
IMG_HEIGHT=180
IMG_WIDTH=180
EPOCH=10
BATCH_SIZE=16
SEED=42
CHANNELS=3

In [5]:
##loading dataset
raw_data=tf.keras.preprocessing.image_dataset_from_directory(
   RAW_DIR,
   image_size=(IMG_HEIGHT, IMG_WIDTH),
   batch_size=BATCH_SIZE,
   shuffle=True 
)

Found 971 files belonging to 2 classes.


In [7]:
raw_data.class_names

['AiArt', 'RealArt']